# Protocol Benchmarks

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import *
from glob import glob as list_files
from pathlib import Path
import re

Formatting Stuff

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
mptcolors = prop_cycle.by_key()['color']
mptcolors[0]
colors = {'PRDT read': mptcolors[0], 'PRDT mixed': mptcolors[1], 'PRDT write': mptcolors[2], 'etcd read': mptcolors[3], 'etcd mixed': mptcolors[4], 'etcd write': mptcolors[5]}

from itertools import cycle
# solid, dotted, dashed, long dash with offsets, dashdotted, dashdotdotted
lines = ['solid',"dotted", "dashed"]
linecycler = cycle(lines)
linestyle = {'PRDT read': next(linecycler), 'PRDT mixed': next(linecycler), 'PRDT write': next(linecycler), 'etcd read':next(linecycler), 'etcd mixed': next(linecycler), 'etcd write': next(linecycler)}
#mylabels = {'etcd client1 cluster3 mixed 20_120': 'ARDT read', 'ARDT mixed': 'ARDT mixed', 'ARDT put': 'ARDT write', 'etcd get': 'etcd read', 'etcd mixed': 'etcd mixed', 'etcd put': 'etcd write'}

In [ ]:
def load_results(path):
    files = list_files(path)
    dfs = []
    for file in files:
        runId = Path(file).name
        system = Path(file).parent.parent.name
        df = pd.read_csv(file, delimiter=";", dtype={'latency': 'float64', 'send-time': 'float64', 'receive-time': 'float64'})
    
        params = runId.split('-')
        
        df["run_config"] = f"{params[0]} {params[1]} {params[2]} {params[3]}"
        try:
            df["num-clients"] = re.match("client(_?)([0-9])", params[0])[2]
        except:
            df["num-clients"] = "undefined"
        try:
            df["num-nodes"] = re.match("cluster(_?)([0-9])", params[1])[2]
        except:
            df["num-nodes"] = "undefined"
        df["mode"] = params[2]
        df["run_id"] = re.match("(.*)\\.csv", params[4])[1]
        df["run"] = runId
        df["system"] = system
        df["index"] = pd.Series(range(0, len(df)))
        df['receive-time'] = df['receive-time'] - df['send-time'].min()
        df['send-time'] = df['send-time'] - df['send-time'].min()
        dfs.append(df)
    
    df = pd.concat(dfs)
    df["latency"] = (df["latency"] / 1000)
    df['send-time'] = df['send-time'] / 1000
    df['receive-time'] = df['receive-time'] / 1000
    df["unit"] = "ms" # fix unit
    
    return df    

In [ ]:
df = load_results("../oopsla-dist-results-hel/*/*/*.csv")
df

In [ ]:
# fix dataframe
df.loc[df['name'] == 'client_1', 'name'] = 'client1'
df.loc[df['run_config'] == 'client_1 cluster3 write 20_120', 'run_config'] = 'write'
df.loc[df['run_config'] == 'client_1 cluster3 mixed 20_120', 'run_config'] = 'mixed'
df.loc[df['run_config'] == 'client_1 cluster3 read 20_120', 'run_config'] = 'read'
df.loc[df['run_config'] == 'client1 cluster3 write 20_120', 'run_config'] = 'write'
df.loc[df['run_config'] == 'client1 cluster3 mixed 20_120', 'run_config'] = 'mixed'
df.loc[df['run_config'] == 'client1 cluster3 read 20_120', 'run_config'] = 'read'
df.loc[df['run_config'] == 'client_1 cluster3 write 0_240', 'run_config'] = 'write'
df.loc[df['run_config'] == 'client_1 cluster3 mixed 0_240', 'run_config'] = 'mixed'
df.loc[df['run_config'] == 'client_1 cluster3 read 0_240', 'run_config'] = 'read'
df.loc[df['run_config'] == 'client1 cluster3 write 0_240', 'run_config'] = 'write'
df.loc[df['run_config'] == 'client1 cluster3 mixed 0_240', 'run_config'] = 'mixed'
df.loc[df['run_config'] == 'client1 cluster3 read 0_240', 'run_config'] = 'read'

df.loc[df['system'] == 'pb', 'system'] = 'PRDT'
df

## Latency Plotting

In [ ]:
latency_df = df[['system', 'mode', 'latency', 'index', 'name']] \
    .groupby(['system', 'mode', 'index', 'name']) \
    .mean().reset_index().set_index('index')

In [ ]:
#latency_df['latency'] = latency_df['latency'] / 130
latency_df

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,5))

def plot_latency(ax, label: str, df: pd.DataFrame, group_size):
    grouped_df = df.groupby({x: (x // group_size) * group_size for x in range(len(df))})
    mean_latency = grouped_df.mean()
    mean_latency.plot(y='latency', ax=ax, label=label, color=colors.get(label, 'pink'), linestyle=linestyle.get(label))
    


for system in latency_df['system'].unique():
    print(system)
    system_df = latency_df[latency_df['system'] == system]
    
    for mode in latency_df['mode'].unique():
        print(mode)
        run_df = system_df[system_df['mode'] == mode]
        plot_latency(ax, f"{system} {mode}", run_df[['latency']], 10)

ax.set_xlabel("number of queries")
ax.set_ylabel("latency [ms]")

ax.set_ylim([0, 136])

#order = [3,4,5,2,1,0]
#ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order])
ax.legend()

plt.savefig(fname="latency.png", format="png")
plt.savefig(fname="latency.pdf", format="pdf")

In [ ]:
for system in latency_df["system"].unique():
    system_df = latency_df[latency_df.system == system]
    for mode in sys_df["mode"].unique():
            filtered_df = sys_df[(sys_df["mode"] == mode)]
            mean_latency = filtered_df["latency"].quantile(0.5)
            print(f"mean latency for {system} {mode}: {mean_latency}")

## Throughput Plotting

In [ ]:
df["seconds"] = (df["receive-time"] / 1000).apply(round)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,5))

res_data = []

for system in df['system'].unique():
    sys_df = df[df['system'] == system]
    
    for run_config in sys_df["run_config"].unique():
        filtered_df = sys_df[(sys_df["run_config"] == run_config)]
        
        throughput = filtered_df.groupby("seconds")["name"].count() / len(sys_df[sys_df['run_config'] == run_config]['run_id'].unique())
        throughput.plot(ax=ax, label=f"{system} {run_config}")
        mean_throughput = throughput.quantile(0.5)
        res_data.append([system, run_config, mean_throughput])
        print(f"Mean Throughput for {system} {run_config}: {mean_throughput}")
    
        # also_mean_throughput = filtered_df["name"].count() / filtered_df["seconds"].max()
        # print(f"Diff {mean_throughput - also_mean_throughput}")
        
        ax.axhline(y=mean_throughput, color=ax.get_lines()[-1].get_color())
        # ax.axhline(y=also_mean_throughput, color=ax.get_lines()[-1].get_color())

ax.legend()
ax.set_ylim(ymin=0)

ax.set_xlabel('time [s]')
ax.set_ylabel('throughput [ops/s]')

plt.savefig(fname="throughput.pdf", format="pdf")
plt.savefig(fname="throughput.png", format="png")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,5))

res_data = []

for system in df['system'].unique():
    sys_df = df[df['system'] == system]
    
    for mode in sys_df["mode"].unique():
        filtered_df = sys_df[(sys_df["mode"] == mode)]
        
        throughput = filtered_df.groupby("seconds")["name"].count() / len(sys_df[sys_df['mode'] == mode]['run_id'].unique())
        label = f"{system} {mode}"
        throughput.plot(ax=ax, label=label, color=colors.get(label, 'pink'), linestyle=linestyle.get(label))
        mean_throughput = throughput.quantile(0.5)
        res_data.append([system, mode, mean_throughput])
        print(f"Mean Throughput for {system} {mode}: {mean_throughput}")
    
        # also_mean_throughput = filtered_df["name"].count() / filtered_df["seconds"].max()
        # print(f"Diff {mean_throughput - also_mean_throughput}")
        
        #ax.axhline(y=mean_throughput, color=ax.get_lines()[-1].get_color())
        # ax.axhline(y=also_mean_throughput, color=ax.get_lines()[-1].get_color())

ax.legend()
ax.set_ylim(ymin=0)

ax.set_xlabel('time [s]')
ax.set_ylabel('throughput [ops/s]')

plt.savefig(fname="throughput.pdf", format="pdf")
plt.savefig(fname="throughput.png", format="png")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,5))

res_df = pd.DataFrame(data=res_data, columns=['system', 'type', 'throughput'])

etcd_df = res_df[res_df.system == 'etcd'][['type', 'throughput']]
pb_df = res_df[res_df.system == 'PRDT'][['type', 'throughput']]

pd.DataFrame(data={'type': etcd_df.type, 'etcd': list(etcd_df.throughput), 'prdt': list(pb_df.throughput)}).set_index('type').plot.bar(ax=ax)

ax.set_ylabel('throughput [ops/s]')
ax.set_xlabel('')

# fig.savefig(fname="throughput_comp.pdf", format="pdf")

In [ ]:
df.groupby(['system', 'run_config', 'run_id'])['send-time'].max() / 1000